In [ ]:
from glass.ete.otol.pg import osm_to_lulc
from glass.prop.prj import get_epsg
import datetime
import os

osm = '/mnt/disk1/jasp/osmlulc/osme/terceira.pbf'

nomenclature = 'clc'

refraster = '/mnt/disk1/jasp/osmlulc/ref/rst_terceira.tif'

result = '/mnt/disk1/jasp/osmlulc/terceira.gpkg'

db = '/mnt/disk1/jasp/osmlulc/terceira_23072023.sql'

ws = '/mnt/disk1/jasp/osmlulc/debug'

time_a = datetime.datetime.now().replace(microsecond=0)

res, logs = osm_to_lulc(
    osm, nomenclature, get_epsg(refraster),
    result, savedb=db, tmpfld=ws
)

time_b = datetime.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [3]:
#shp = '/mnt/d/osm2lulc02/lulc_fmosm/terceira.gpkg'
shp = '/mnt/disk1/jasp/osmlulc/terceira.gpkg'

refraster = '/mnt/disk1/jasp/osmlulc/ref/rst_terceira.tif'

lyr = 'osmtolulc_v2'

col = 'lulc'

osm_db = 'terceira'

In [1]:
import datetime
import copy

from glass.cons.otol import classes_priority, OTOL_GEOM
from glass.sql.q import q_to_obj
from glass.prop.sql import row_num
from glass.gp.ovl.sql import st_erase, st_erase_opt
from glass.it.db import gpkg_lyr_attr_to_psql
from glass.gp.seg.sql import geomseg_to_newtbl
from glass.prop.prj import get_epsg

In [4]:
epsg = get_epsg(refraster)

In [5]:
order_cls = classes_priority(2)

In [6]:
order_cls

[{'fid': 87, 'bigbbox': '1'},
 {'fid': 86, 'bigbbox': '1'},
 {'fid': 29, 'bigbbox': 0},
 {'fid': 42, 'bigbbox': '1'},
 {'fid': 41, 'bigbbox': '1'},
 {'fid': 26, 'bigbbox': 0},
 {'fid': 103, 'bigbbox': 0},
 {'fid': 89, 'bigbbox': 0},
 {'fid': 102, 'bigbbox': 0},
 {'fid': 30, 'bigbbox': 0},
 {'fid': 28, 'bigbbox': 0},
 {'fid': 33, 'bigbbox': 0},
 {'fid': 34, 'bigbbox': 0},
 {'fid': 32, 'bigbbox': 0},
 {'fid': 35, 'bigbbox': 0},
 {'fid': 24, 'bigbbox': 0},
 {'fid': 70, 'bigbbox': 0},
 {'fid': 67, 'bigbbox': 0},
 {'fid': 38, 'bigbbox': 0},
 {'fid': 36, 'bigbbox': 0}]

In [ ]:
table_cls = gpkg_lyr_attr_to_psql(shp, lyr, col, osm_db, 'tblcls')

In [ ]:
# Create segments table if necessary
for cls in order_cls:
    if not cls['bigbox']: continue

    if cls['fid'] not in table_cls: continue
    
    table_cls[cls['fid']] = geomseg_to_newtbl(
        osm_db, table_cls[cls['fid']],
        'fid', 'geom', 'polygon', epsg, f"{table_cls[cls['fid']]}_seg",
        cols={'lulc' : 'integer', 'leg': 'text'}, subdivide_factor=10
    )

In [ ]:
table_cls

In [ ]:
refname = copy.deepcopy(table_cls)

for e in range(len(order_cls)):
    if e + 1 == len(order_cls): break

    if order_cls[e]['fid'] not in table_cls: continue

    for i in range(e+1, len(order_cls)):
        if order_cls[i]['fid'] not in table_cls: continue

        time_a = datetime.datetime.now().replace(microsecond=0)

        table_cls[order_cls[i]] = st_erase_opt(
            osm_db,
            table_cls[order_cls[i]['fid']], 'fid',
            table_cls[order_cls[e]['fid']],
            "geom", "geom",
            otbl=f"{refname[order_cls[i]['fid']]}_{str(e)}"
        )

        time_b = datetime.datetime.now().replace(microsecond=0)

        print(f'{table_cls[order_cls[i]['fid']]} <-> {table_cls[order_cls[e]['fid']]} || {time_b - time_a}')
        print('---------------------------------')

        nrows = row_num(osm_db, table_cls[order_cls[i]['fid']], api='psql')

        if not nrows:
            del table_cls[order_cls[i]['fid']]
            continue

        # Create Geometry index for the new table
        qs = [(
            f"ALTER TABLE {table_cls[order_cls[i]['fid']]} ADD CONSTRAINT "
            f"{table_cls[order_cls[i]['fid']]}_pk PRIMARY KEY (fid)"
        ), (
            f"CREATE INDEX {table_cls[order_cls[i]['fid']]}_geom_idx ON "
            f"{table_cls[order_cls[i]['fid']]} "
            f"USING gist ({OTOL_GEOM})"
        )]

In [ ]:
order_cls

In [ ]:
table_cls

In [ ]:
time_a = datetime.datetime.now().replace(microsecond=0)

ntbl = st_erase(osm_db, "tblcls_28", "tblcls_86", "geom", "geom", otbl="metodo_1")

time_b = datetime.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [ ]:
time_a = datetime.datetime.now().replace(microsecond=0)

ntbl = st_erase(osm_db, "tblcls_28", "tblcls_86", "geom", "geom", otbl="metodo_2", method=2)

time_b = datetime.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [ ]:
time_a = datetime.datetime.now().replace(microsecond=0)

ntbl = st_erase(osm_db, "tblcls_28", "tblcls_86", "geom", "geom", otbl="metodo_3", method=3)

time_b = datetime.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [ ]:
from glass.ete.otol.tools import module_osmtags

tags = module_osmtags(2)

In [ ]:
tags[tags.name == 'roads'].lulc_id.unique()

In [ ]:
tags.name.unique()

In [ ]:
tags[
    (tags['value'] == 'yes') &
    (tags['key'] == 'building')
]

In [ ]:
from glass.ete.osm2lulc.grs import osm_to_lulc
from glass.ete.osm2lulc.utils import record_time_consumed
import datetime
import os
from glass.pys.oss import fprop, mkdir
from glass.pys.tm import now_as_str

In [ ]:
time_a = datetime.datetime.now().replace(microsecond=0)

ws = mkdir(os.path.join(
    os.path.dirname(osm),
    fprop(result, 'fn')
), overwrite=True)

shp, time_check = osm_to_lulc(
    osm, nomenclature, refraster,
    result, tmpfld=ws,
    overwrite=True, savedb=db
)

time_b = datetime.datetime.now().replace(microsecond=0)
    
# Record time consumed in xlsx table
record_time_consumed(time_check, os.path.join(
    os.path.dirname(result),
    fprop(result, 'fn') + '.xlsx'
))

print(time_b - time_a)